In [1]:
from precomputed_python import AnnotationReader, morton_code_to_gridpt, compressed_morton_code
import numpy as np
import pandas as pd
import asyncio
import tensorstore as ts
from neuroglancer import viewer_state

In [2]:
# pre_ann=AnnotationReader("gs://neuroglancer-20191211_fafbv14_buhmann2019_li20190805")
# ann = pre_ann.get_by_id(233446588)
pre_ann = AnnotationReader("gs://allen_neuroglancer_ccf/mesoscale_connectivity_atlas_points_v9")
# pre_ann = AnnotationReader("gs://allen_neuroglancer_ccf/merscope_ccffix_1")


I0401 06:42:51.988469 7496059 google_auth_provider.cc:149] Using credentials at /Users/forrestc/.config/gcloud/application_default_credentials.json
I0401 06:42:51.989955 7496059 google_auth_provider.cc:156] Using OAuth2 AuthProvider


In [3]:
pre_ann.get_relationships()

['rel_structure_id', 'rel_experiment_id']

In [4]:
pre_ann.get_property_names()

['injection_volume',
 'transgenic_line',
 'structure_abbrev',
 'structure_name',
 'gender',
 'structure_color']

In [5]:
pre_ann.properties

[AnnotationPropertySpec({"description": "injection_volume", "id": "injection_volume", "type": "float32"}),
 AnnotationPropertySpec({"description": "transgenic_line", "enum_labels": ["nan", "Rbp4-Cre_KL100", "Slc6a4-Cre_ET33", "Vipr2-Cre_KE2", "Rasgrf2-T2A-dCre", "Emx1-IRES-Cre", "Gpr26-Cre_KO250", "Slc6a4-CreERT2_EZ13", "Erbb4-T2A-CreERT2", "Trib2-F2A-CreERT2", "Fezf1-T2A-dCre", "Efr3a-Cre_NO108", "Slc17a6-IRES-Cre", "Cux2-IRES-Cre", "Esr1-2A-Cre", "Ppp1r17-Cre_NL146", "Slc6a5-Cre_KF109", "Ntng2-IRES2-Cre", "Cart-Tg1-Cre", "Gnb4-IRES2-Cre", "Glt25d2-Cre_NF107", "Sim1-Cre_KJ18", "Grik4-Cre", "Cck-IRES-Cre", "Grm2-Cre_MR90", "Th-Cre_FI172", "Plxnd1-Cre_OG1", "Slc18a2-Cre_OZ14", "Slc6a3-Cre", "Syt17-Cre_NO14", "Prkcd-GluCla-CFP-IRES-Cre", "Slc32a1-IRES-Cre", "Adcyap1-2A-Cre", "Cnnm2-Cre_KD18", "Ntrk1-IRES-Cre", "Satb2-Cre_MO23", "A930038C07Rik-Tg1-Cre", "Calb1-T2A-dgCre", "Tlx3-Cre_PL56", "Chat-IRES-Cre-neo", "Drd1a-Cre_EY262", "Grp-Cre_KH288", "Lypd6-Cre_KL156", "Kcng4-Cre", "Tac1-IRES2-

In [6]:
pre_ann.coordinate_space

CoordinateSpace({'x': [np.float64(1e-06), 'm'], 'y': [np.float64(1e-06), 'm'], 'z': [np.float64(1e-06), 'm']})

In [7]:
all_ids = pre_ann.get_all_annotation_ids()
all_ids

array([100147785, 100147853, 100147861, ..., 666909423, 671463542,
       671464291], dtype='>u8')

In [8]:
ann=pre_ann.get_by_id(all_ids[0])
ann

{'injection_volume': np.float32(0.1759717),
 'transgenic_line': np.uint16(0),
 'structure_abbrev': np.uint16(54),
 'structure_name': np.uint16(54),
 'gender': np.uint16(0),
 'structure_color': array([255, 144, 159], dtype=uint8),
 'padding15': array([0], dtype=uint8),
 'structure_id': array([1020], dtype=uint64),
 'experiment_id': array([100147785], dtype=uint64),
 'point': array([7600., 3380., 7070.], dtype=float32)}

In [9]:
df =pre_ann.get_by_relationship('structure_id', ann['structure_id'][0])
df

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,point
ID,,,,,,,,
267999740,0.302530,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7540.0, 2400.0, 6990.0]"
183011353,0.200614,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7340.0, 3550.0, 6870.0]"
174781014,0.166800,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7440.0, 3600.0, 6920.0]"
100147785,0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7600.0, 3380.0, 7070.0]"
292212456,0.039326,Grik4-Cre,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],"[7560.0, 3450.0, 7110.0]"
182515576,0.045167,Gal-Cre_KI87,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[6830.0, 3760.0, 7370.0]"
182892855,0.025026,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7230.0, 3980.0, 7070.0]"
180708524,0.015272,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],"[7350.0, 3930.0, 7040.0]"
301674988,0.009943,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],"[7430.0, 4110.0, 7440.0]"


In [10]:
df =pre_ann.get_by_relationship('structure_id', ann['structure_id'][0], get_relationships=True)
df

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,structure_id,experiment_id,point
ID,,,,,,,,,,
267999740,0.302530,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[267999740],"[7540.0, 2400.0, 6990.0]"
183011353,0.200614,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[183011353],"[7340.0, 3550.0, 6870.0]"
174781014,0.166800,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[174781014],"[7440.0, 3600.0, 6920.0]"
100147785,0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[100147785],"[7600.0, 3380.0, 7070.0]"
292212456,0.039326,Grik4-Cre,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],[1020],[292212456],"[7560.0, 3450.0, 7110.0]"
182515576,0.045167,Gal-Cre_KI87,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[182515576],"[6830.0, 3760.0, 7370.0]"
182892855,0.025026,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[182892855],"[7230.0, 3980.0, 7070.0]"
180708524,0.015272,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[180708524],"[7350.0, 3930.0, 7040.0]"
301674988,0.009943,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],[1020],[301674988],"[7430.0, 4110.0, 7440.0]"


In [10]:
df = pre_ann.get_all_annotations()
df.head()

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,structure_id,experiment_id,point
ID,,,,,,,,,,
100147785,0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[100147785],"[7600.0, 3380.0, 7070.0]"
100147853,0.073548,nan,VISp,Primary visual area,M,"[8, 133, 140]",[0],[385],[100147853],"[9440.0, 1220.0, 8630.0]"
100147861,0.138880,nan,CA1,Field CA1,M,"[126, 208, 75]",[0],[382],[100147861],"[6640.0, 1650.0, 7510.0]"
100148142,0.122178,nan,RSPv,"Retrosplenial area, ventral part",M,"[26, 166, 152]",[0],[886],[100148142],"[6830.0, 1260.0, 6160.0]"
100148143,0.114309,nan,DG,Dentate gyrus,M,"[126, 208, 75]",[0],[726],[100148143],"[6690.0, 2690.0, 6190.0]"


In [15]:
some_ids = df.index[0:10]
pre_ann.get_by_ids(some_ids)

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,structure_id,experiment_id,point
ID,,,,,,,,,,
267999740,0.302530,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[267999740],"[7540.0, 2400.0, 6990.0]"
183011353,0.200614,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[183011353],"[7340.0, 3550.0, 6870.0]"
174781014,0.166800,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[174781014],"[7440.0, 3600.0, 6920.0]"
100147785,0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[100147785],"[7600.0, 3380.0, 7070.0]"
292212456,0.039326,Grik4-Cre,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],[1020],[292212456],"[7560.0, 3450.0, 7110.0]"
182515576,0.045167,Gal-Cre_KI87,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[182515576],"[6830.0, 3760.0, 7370.0]"
182892855,0.025026,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[182892855],"[7230.0, 3980.0, 7070.0]"
180708524,0.015272,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[180708524],"[7350.0, 3930.0, 7040.0]"
301674988,0.009943,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0],[1020],[301674988],"[7430.0, 4110.0, 7440.0]"


In [11]:
pre_ann.read_annotations_in_chunk('spatial0', chunk_index=[5,3,5])

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,point
ID,,,,,,,,
183225830,0.236394,Grm2-Cre_MR90,PVT,Paraventricular nucleus of the thalamus,F,"[255, 144, 159]",[0],"[6060.0, 4010.0, 5510.0]"
125830911,0.268019,nan,LSr,"Lateral septal nucleus, rostral (rostroventral...",M,"[144, 203, 237]",[0],"[5510.0, 3250.0, 5690.0]"
182341627,0.041936,Syt6-Cre_KI148,TRS,Triangular nucleus of septum,F,"[150, 167, 211]",[0],"[5350.0, 4050.0, 6040.0]"
268399145,0.050636,Ppp1r17-Cre_NL146,MD,Mediodorsal nucleus of thalamus,F,"[255, 144, 159]",[0],"[6040.0, 3580.0, 6040.0]"
287950390,0.046494,Calb2-IRES-Cre,TRS,Triangular nucleus of septum,F,"[150, 167, 211]",[0],"[5510.0, 3560.0, 6040.0]"
178486024,0.040623,Cart-Tg1-Cre,LSr,"Lateral septal nucleus, rostral (rostroventral...",F,"[144, 203, 237]",[0],"[5140.0, 3570.0, 6140.0]"
